# 01 Image Preprocessing - PART 03 - PREPARATION

### Imports

In [13]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random
from sys import getsizeof
from tensorflow import keras

### Mode of running

In [14]:
TEST_MODE = False

### Paths

In [15]:
BASE_PATH = './data'
ORIGINAL_IMAGES = f'{BASE_PATH}/unsplit'

In [16]:
CATEGORIES = os.listdir(ORIGINAL_IMAGES) # list the names of the categories that in my data
print(str(len(CATEGORIES)),'CATEGORIES are ', CATEGORIES)

8 CATEGORIES are  ['Black_Hooded_Oriole', 'Common_Myna', 'Indian_Peafowl', 'Intermediate_Egret', 'Other', 'Red_Faced_Malkoha', 'Rose_Ringed_Parakeet', 'White_Throated_Kingfisher']


### Reading the augmented images

In [17]:
def read_images():
    dataset = []
    
    for category in CATEGORIES:
        cat_path = os.path.join(ORIGINAL_IMAGES, category)
        cat_num = CATEGORIES.index(category)
        for img_filename in os.listdir(cat_path):
            img = cv2.imread(os.path.join(cat_path, img_filename))[...,::-1]  # read & convert from BGR to RGB
            dataset.append([img, cat_num])
        if TEST_MODE and cat_num == 2:
            break # - then will only take the images from the first category
            
    return dataset

In [ ]:
dataset = read_images()
len(dataset)

In [ ]:
dataset[0][0].shape

In [ ]:
plt.imshow(dataset[0][0][:,:])

In [28]:
mem_err_avoid_batch_size = 100
itr = 0
start = itr*mem_err_avoid_batch_size
end = np.min((itr+1)*mem_err_avoid_batch_size + 1, len(aug_dataset))

AxisError: axis 2834 is out of bounds for array of dimension 0

In [ ]:
resized_aug_dataset = []

for aug_data in aug_dataset:
    resized_aug_dataset.append(aug_data[0]/255.)

In [ ]:
resized_aug_dataset

In [ ]:
plt.imshow(aug_dataset[0][0])

In [22]:
random.shuffle(training_data)

NameError: name 'training_data' is not defined